In [23]:
# 데이터 출처: 문화체육관광부 문화 공공데이터 광장 - KOBIS-영화목록
# 최근 갱신일: 2023-10-24
# 링크: https://www.culture.go.kr/data/filedat/filedatDtl.do?fileDataNo=00000000000000000184&category=C&orderBy=dwldCnt&category=C&category=C

In [24]:
import pandas as pd

df = pd.read_csv('영화진흥위원회_KOBIS-영화목록.csv', encoding='UTF-8', low_memory=False)

df.head()

,RNUM,PUBLISHER,COLLECTIONDB,CREATOR,TITLE,ALTERNATIVETITLE,ISSUEDDATE,MODIFIEDDATE,SUBJECT,SUBJECTKEYWORD,...,EXTENT,PERSON,RIGHTS,CONTRIBUTOR,GRADE,AFFILIATION,MEDIUM,STATE,PERIOD,TABLEOFCONTENTS
0,1,B551004,kobis04영화목록,누리집,늦더위,Late Summer,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,B551004,kobis04영화목록,누리집,늦봄,late spring,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,B551004,kobis04영화목록,누리집,늦어도 그날까지,Until the day,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,B551004,kobis04영화목록,누리집,늦여름,MeinlangsamesLeben,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,B551004,kobis04영화목록,누리집,늦여름,Late Summer,NaN,NaN,C,NaN,...,NaN,NaN,문경법,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# SUBJECTCATEGORY 필드에 '성인물', '에로' 글자가 포함된 행 제외
filtered_df = df[~df['SUBJECTCATEGORY'].str.contains('성인물|에로', na=False)]

# TITLE 필드에 '발정', '섹스', '불륜', '음란', '유부녀', '신음', '오르가즘' 글자가 포함된 행 제외
filtered_df = filtered_df[~filtered_df['TITLE'].str.contains('발정|섹스|불륜|음란|유부녀|신음|오르가즘', na=False)]


In [26]:
print(len(df))
print(len(filtered_df))

83280
78639


In [27]:
# TITLE, ALTERNATIVETITLE, UCI, URL, CREATEDDATE, SPATIALCOVERAGE, SUBJECTCATEGORY, RIGHTS 필드만 추출
selected_columns = ['TITLE', 'ALTERNATIVETITLE', 'UCI', 'URL', 'CREATEDDATE', 'SPATIALCOVERAGE', 'SUBJECTCATEGORY', 'RIGHTS']
extracted_df = filtered_df[selected_columns]

extracted_df = extracted_df.copy()
extracted_df['CREATEDDATE'] = extracted_df['CREATEDDATE'].astype('Int64')

extracted_df.head()

,TITLE,ALTERNATIVETITLE,UCI,URL,CREATEDDATE,SPATIALCOVERAGE,SUBJECTCATEGORY,RIGHTS
0,늦더위,Late Summer,G706+KOBIS04-MV.list.0020111002,http://www.kobis.or.kr/kobis/business/mast/mvi...,2010,한국,드라마,NaN
1,늦봄,late spring,G706+KOBIS04-MV.list.0020184358,http://www.kobis.or.kr/kobis/business/mast/mvi...,2017,한국,드라마,NaN
2,늦어도 그날까지,Until the day,G706+KOBIS04-MV.list.0019698040,http://www.kobis.or.kr/kobis/business/mast/mvi...,1969,NaN,NaN,NaN
3,늦여름,MeinlangsamesLeben,G706+KOBIS04-MV.list.0020018831,http://www.kobis.or.kr/kobis/business/mast/mvi...,2001,NaN,NaN,NaN
4,늦여름,Late Summer,G706+KOBIS04-MV.list.0020138935,http://www.kobis.or.kr/kobis/business/mast/mvi...,2012,한국,드라마,문경법


In [28]:
extracted_df.columns = extracted_df.columns.str.lower()

extracted_df.to_csv('output.csv', index=False)

In [2]:
import sqlalchemy as sa


def pg_connect(user, password, db, host, port=5432):
    url = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = sa.create_engine(url, client_encoding='utf8')
    return engine

user = 'xxxx'
password = 'xxxx'
db = 'xxxx'
host = 'xxxx'
port = 9999

engine = pg_connect(user, password, db, host)


In [35]:
extracted_df.to_sql(name='temp_table', con=engine, if_exists='replace', index=False)

639

In [3]:
insert_sql = """
INSERT INTO movie (title, "alternativeTitle", uci, url, year, region, category, rights)
SELECT 
    title,
    alternativetitle,
    uci,
    url,
    createddate,
    spatialcoverage,
    subjectcategory,
    rights
FROM temp_table
"""
 
# FIXME: why not working?
# pgAdmin 콘솔 상에서 위 커맨드 복붙하면 들어갑니다.
with engine.connect() as con:
    con.execute(sa.text(insert_sql))

engine.dispose()